## Кривоногов Н.В., NLP, практическое задание № 8

"На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN  и (RNN -> CNN)
4. сдлать выводы что получилось лучше"

https://drive.google.com/file/d/1iO8kRTY0N_lCrv87BuXTTN7RMRbKirTD/view?usp=sharing

In [1]:
! gdown 1iO8kRTY0N_lCrv87BuXTTN7RMRbKirTD

Downloading...
From: https://drive.google.com/uc?id=1iO8kRTY0N_lCrv87BuXTTN7RMRbKirTD
To: /content/GB_NLP_HW_8.zip
100% 12.7M/12.7M [00:00<00:00, 45.2MB/s]


In [2]:
! unzip -o /content/GB_NLP_HW_8.zip

Archive:  /content/GB_NLP_HW_8.zip
  inflating: test.csv                
  inflating: train.csv               
  inflating: val.csv                 


In [3]:
! pip install stop_words

In [4]:
! pip install pymorphy2

In [5]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")
df_val = pd.read_csv("/content/val.csv")

In [6]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [7]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [8]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [9]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [10]:
tokenizer = Tokenizer(num_words=None,
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [11]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [12]:
# Training
epochs = 10
batch_size = 512

In [13]:
comparing_list = []

1. построить свёрточные архитектуры

In [14]:
model_name = 'CNN'

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
#model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [15]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    )

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 27, 30)            7743240   
                                                                 
 conv1d (Conv1D)             (None, 25, 128)           11648     
                                                                 
 activation (Activation)     (None, 25, 128)           0         
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                        

In [16]:
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 47s 142ms/step - loss: 0.5700 - accuracy: 0.6900 - val_loss: 0.4940 - val_accuracy: 0.7536
Epoch 2/10
319/319 [==============================] - 46s 143ms/step - loss: 0.3357 - accuracy: 0.8612 - val_loss: 0.5332 - val_accuracy: 0.7488


In [17]:
score = model.evaluate(X_valid, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Val loss:', score[0])
print('Val accuracy:', score[1])

45/45 [==============================] - 0s 10ms/step - loss: 0.5839 - accuracy: 0.7345


Val loss: 0.5838587284088135
Val accuracy: 0.7345148324966431


In [18]:
comparing_list.append([model_name, score[1]])

2. построить различные архитектуры с RNN

2.1. SimpleRNN

In [19]:
model_name = 'SimpleRNN'

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
#model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [20]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    )

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                6080      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 7,753,545
Trainable params: 7,753,545
Non-trainable params: 0
_________________________________________________________________


In [21]:
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 52s 159ms/step - loss: 0.5549 - accuracy: 0.7045 - val_loss: 0.4912 - val_accuracy: 0.7566
Epoch 2/10
319/319 [==============================] - 49s 155ms/step - loss: 0.2780 - accuracy: 0.8868 - val_loss: 0.5587 - val_accuracy: 0.7463


In [22]:
score = model.evaluate(X_valid, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Val loss:', score[0])
print('Val accuracy:', score[1])

45/45 [==============================] - 1s 18ms/step - loss: 0.5883 - accuracy: 0.7319


Val loss: 0.5883012413978577
Val accuracy: 0.7318696975708008


In [23]:
comparing_list.append([model_name, score[1]])

2.2. LSTM

In [24]:
model_name = 'LSTM'

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
#model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [25]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    )

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 lstm (LSTM)                 (None, 64)                24320     
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 7,771,785
Trainable params: 7,771,785
Non-trainable params: 0
_________________________________________________________________


In [26]:
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 112s 339ms/step - loss: 0.5498 - accuracy: 0.7091 - val_loss: 0.4885 - val_accuracy: 0.7594
Epoch 2/10
319/319 [==============================] - 101s 316ms/step - loss: 0.3074 - accuracy: 0.8742 - val_loss: 0.5387 - val_accuracy: 0.7450


In [27]:
score = model.evaluate(X_valid, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Val loss:', score[0])
print('Val accuracy:', score[1])

45/45 [==============================] - 2s 48ms/step - loss: 0.5735 - accuracy: 0.7365


Val loss: 0.5734942555427551
Val accuracy: 0.7365427613258362


In [28]:
comparing_list.append([model_name, score[1]])

2.3. GRU

In [29]:
model_name = 'GRU'

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
#model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [30]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    )

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 gru (GRU)                   (None, 64)                18432     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 7,765,897
Trainable params: 7,765,897
Non-trainable params: 0
_________________________________________________________________


In [31]:
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 96s 288ms/step - loss: 0.5505 - accuracy: 0.7103 - val_loss: 0.4946 - val_accuracy: 0.7531
Epoch 2/10
319/319 [==============================] - 88s 277ms/step - loss: 0.3074 - accuracy: 0.8750 - val_loss: 0.5425 - val_accuracy: 0.7465


In [32]:
score = model.evaluate(X_valid, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Val loss:', score[0])
print('Val accuracy:', score[1])

45/45 [==============================] - 2s 37ms/step - loss: 0.5772 - accuracy: 0.7372


Val loss: 0.5771825313568115
Val accuracy: 0.7372040748596191


In [33]:
comparing_list.append([model_name, score[1]])

3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)

3.1. CNN -> RNN

In [34]:
model_name = 'CNN -> RNN'

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
#model.add(Masking(mask_value=0.0))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D(keepdims=True))

model.add(LSTM(128, recurrent_dropout=0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [35]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    )

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 conv1d_1 (Conv1D)           (None, 25, 128)           11648     
                                                                 
 activation_1 (Activation)   (None, 25, 128)           0         
                                                                 
 global_max_pooling1d_1 (Glo  (None, 1, 128)           0         
 balMaxPooling1D)                                                
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                      

In [36]:
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 51s 150ms/step - loss: 0.5644 - accuracy: 0.6918 - val_loss: 0.4914 - val_accuracy: 0.7519
Epoch 2/10
319/319 [==============================] - 48s 151ms/step - loss: 0.3092 - accuracy: 0.8724 - val_loss: 0.5470 - val_accuracy: 0.7476


In [37]:
score = model.evaluate(X_valid, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Val loss:', score[0])
print('Val accuracy:', score[1])

45/45 [==============================] - 1s 14ms/step - loss: 0.6191 - accuracy: 0.7355


Val loss: 0.6190803647041321
Val accuracy: 0.735528826713562


In [38]:
comparing_list.append([model_name, score[1]])

3.2. RNN -> CNN

In [39]:
model_name = 'RNN -> CNN'

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
#model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2, return_sequences="True"))

model.add(Conv1D(64, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [40]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
    )

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 27, 30)            7743240   
                                                                 
 lstm_2 (LSTM)               (None, 27, 64)            24320     
                                                                 
 conv1d_2 (Conv1D)           (None, 25, 64)            12352     
                                                                 
 activation_2 (Activation)   (None, 25, 64)            0         
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                      

In [41]:
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 114s 347ms/step - loss: 0.5547 - accuracy: 0.7026 - val_loss: 0.4892 - val_accuracy: 0.7571
Epoch 2/10
319/319 [==============================] - 107s 336ms/step - loss: 0.3057 - accuracy: 0.8745 - val_loss: 0.5445 - val_accuracy: 0.7440


In [42]:
score = model.evaluate(X_valid, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Val loss:', score[0])
print('Val accuracy:', score[1])

45/45 [==============================] - 2s 53ms/step - loss: 0.5782 - accuracy: 0.7350


Val loss: 0.5781680941581726
Val accuracy: 0.7349997758865356


In [43]:
comparing_list.append([model_name, score[1]])

4. сделать выводы, что получилось лучше

In [44]:
comparing_list

[['CNN', 0.7345148324966431],
 ['SimpleRNN', 0.7318696975708008],
 ['LSTM', 0.7365427613258362],
 ['GRU', 0.7372040748596191],
 ['CNN -> RNN', 0.735528826713562],
 ['RNN -> CNN', 0.7349997758865356]]

In [46]:
pd.DataFrame(comparing_list, columns=['model_name', 'accuracy']).sort_values(by='accuracy', ascending=False)

,model_name,accuracy
3,GRU,0.737204
2,LSTM,0.736543
4,CNN -> RNN,0.735529
5,RNN -> CNN,0.735000
0,CNN,0.734515
1,SimpleRNN,0.731870


В целом у всех моделей метрика приблизительно равна и колеблется около 0.73.

Лучший результат у Gated Recurrent Unit (GRU).